In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the CycleGAN Generator (simplified version)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# Define the U-Net Model
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Dataset Loader
class PairedImageDataset(Dataset):
    def __init__(self, reflected_folder_path, clear_folder_path, transform=None):
        self.reflected_folder_path = reflected_folder_path
        self.clear_folder_path = clear_folder_path
        self.reflected_image_paths = [os.path.join(reflected_folder_path, file) for file in os.listdir(reflected_folder_path)]
        self.clear_image_paths = [os.path.join(clear_folder_path, file) for file in os.listdir(clear_folder_path)]
        assert len(self.reflected_image_paths) == len(self.clear_image_paths), "The number of reflected and clear images must be the same."
        self.transform = transform

    def __len__(self):
        return len(self.reflected_image_paths)

    def __getitem__(self, idx):
        reflected_img = Image.open(self.reflected_image_paths[idx]).convert('RGB')
        clear_img = Image.open(self.clear_image_paths[idx]).convert('RGB')
        if self.transform:
            reflected_img = self.transform(reflected_img)
            clear_img = self.transform(clear_img)
        return reflected_img, clear_img

# Function to save images
def save_images(image, path):
    save_image(image, path)
    print(f"Image saved to: {path}")

# Define the training function
def train_model(generator, unet, dataloader, num_epochs=5, lr=0.0002):
    criterion = nn.MSELoss()
    optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_unet = optim.Adam(unet.parameters(), lr=lr, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for i, (img_with_reflection, img_clear) in enumerate(dataloader):
            img_with_reflection, img_clear = img_with_reflection.to(device), img_clear.to(device)

            # Step 1: Train the CycleGAN generator
            generator.train()
            fake_image = generator(img_with_reflection)
            loss_g = criterion(fake_image, img_clear)  # Rough alignment loss
            optimizer_g.zero_grad()
            loss_g.backward()
            optimizer_g.step()

            # Step 2: Train the U-Net
            unet.train()
            refined_image = unet(fake_image.detach())
            loss_unet = criterion(refined_image, img_clear)  # Fine alignment loss
            optimizer_unet.zero_grad()
            loss_unet.backward()
            optimizer_unet.step()

            if i % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], "
                      f"Loss_G: {loss_g.item():.4f}, Loss_UNet: {loss_unet.item():.4f}")

# Display results function
def display_results(image_path, generator, unet):
    # Load the image
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    img = Image.open(image_path).convert("RGB")
    input_image = transform(img).unsqueeze(0).to(device)

    # Generate output from CycleGAN
    generator.eval()
    fake_image = generator(input_image).detach()

    # Refine output using U-Net
    unet.eval()
    refined_image = unet(fake_image).detach()

    # Denormalize images for saving and visualization
    def denormalize(tensor):
        return tensor * 0.5 + 0.5  # Reverts normalization (mean=0.5, std=0.5)

    input_image_denorm = denormalize(input_image.squeeze(0)).cpu()
    fake_image_denorm = denormalize(fake_image.squeeze(0)).cpu()
    refined_image_denorm = denormalize(refined_image.squeeze(0)).cpu()

    # Save images
    output_path = r"C:\Users\aryan\OneDrive\Desktop\IMAGE SAVING"
    os.makedirs(output_path, exist_ok=True)
    save_images(input_image_denorm, os.path.join(output_path, "input_image.png"))
    save_images(fake_image_denorm, os.path.join(output_path, "fake_image.png"))
    save_images(refined_image_denorm, os.path.join(output_path, "refined_image.png"))

    # Plot the results
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(input_image_denorm.permute(1, 2, 0))
    axes[0].set_title("Input Image (with reflections)")
    axes[0].axis("off")

    axes[1].imshow(fake_image_denorm.permute(1, 2, 0))
    axes[1].set_title("CycleGAN Output")
    axes[1].axis("off")

    axes[2].imshow(refined_image_denorm.permute(1, 2, 0))
    axes[2].set_title("Final Output (U-Net Refined)")
    axes[2].axis("off")

    plt.show()

# Main function
def main():
    reflected_folder_path = r"C:\Users\aryan\OneDrive\Desktop\IIT BOMBAY\TEST FOR RAW IMAGE"
    clear_folder_path = r"C:\Users\aryan\OneDrive\Desktop\original image"

    # Define the transformation for the images
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Create the paired dataset
    dataset = PairedImageDataset(reflected_folder_path, clear_folder_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    # Initialize models
    generator = Generator().to(device)
    unet = UNet().to(device)

    # Train models
    train_model(generator, unet, dataloader, num_epochs=10, lr=0.0002)

    # Save models
    torch.save(generator.state_dict(), "cycleGAN_generator.pth")
    torch.save(unet.state_dict(), "unet_model.pth")
    print("Training complete and models saved.")

    # Display results on a sample image
    sample_image_path = r"C:\Users\aryan\OneDrive\Desktop\original image\TESTO FOR ORIGINAL IMAGE.png"
    display_results(sample_image_path, generator, unet)

# Run the main function
if __name__ == "__main__":
    main()
